# Seed analysis
Here, the data gathered for the inspecting the model's sensitivity to stochastics will be analyzed.

In [1]:
import pandas as pd
import os
from model.enums import BeliefUpdate
from dmdu.utils_dmdu import make_sure_path_exists, calculate_quantiles

In [2]:
# Parameters
considered_quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
belief_update_fns = [BeliefUpdate.SIT]  # TODO: When data for all bufns: delete this line


In [3]:
# Quantile Data // calculate it and save it
for belief_update_fn in belief_update_fns:  # TODO: When data for all bufns: belief_update_fns -> list(BeliefUpdate)
    # Loading data
    path = dir_path = os.path.join(os.getcwd(), 'data', 'seedanalysis', belief_update_fn.name, 'outcomes.csv')
    outcomes = pd.read_csv(path)
    outcomes = outcomes.rename(columns={'Unnamed: 0': 'uncert_sample'})
    outcome_names = [o for o in outcomes.columns if o not in ['uncert_sample', 'seed']]

    for outcome in outcome_names:
        quantile_data = calculate_quantiles(outcomes_data=outcomes, outcome=outcome, quantiles=considered_quantiles)

        # Save resulting data into csv
        dir_path = os.path.join(os.getcwd(), 'data', 'seedanalysis', belief_update_fn.name, 'quantiles')
        make_sure_path_exists(dir_path)
        file_name = str(outcome) + '.csv'
        path = os.path.join(dir_path, file_name)
        quantile_data.to_csv(path)


In [11]:
# TODO: Plotting of the data
# - read-in the right data from the right files
# - combine the data as needed
#       - quartiles: median and interquartile range
#           1 plot per objective, incl. for each bufn median line and interquartile envelope
#       - 10-and-90%ile?